In [1]:
# !pip install -r requirements.txt
import importlib
importlib.reload
# Load the TensorBoard notebook extension
%load_ext tensorboard
import torch, os, sys
print("torch version  :", torch.__version__)

torch version  : 2.9.0.dev20250715


In [ ]:
!python scripts/convert_dm4.py --input data/Diffraction_SI.dm4 --output data/train_tensor.h5 --downsample 16 --mode bin 

Saved 4788 patterns of size 32*32 → data/train_tensor.pt


In [4]:
!python -m scripts.train --data data/train_tensor.pt --epochs 1 --batch 32 --latent 8 --lr 0.00003 --output_dir outputs --device mps --summary True

Using device: mps
Seed set to 42
Detected input size: 32x32
/Users/louisg/.pyenv/versions/custom_ae/lib/python3.11/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
────────────────────────────────────────────────────────────────────────────────
Layer                              Input → Output                    Params
────────────────────────────────────────────────────────────────────────────────
000_model_encoder_conv_input_Conv2d(1, 1, 32, 32) → (1, 64, 32, 32)       640
001_model_encoder_bn_input_BatchNorm2d(1, 64, 32, 32) → (1, 64, 32, 32)       128
002_model_encoder_conv_pre_Conv2d  (1, 64, 32, 32) → (1, 128, 32, 32)    73,856
003_model_encoder_bn_pre_BatchNorm2d(1, 128, 32, 32) → (1, 128, 32, 32)       256
004_model_encoder_resnet1_conv_block_conv1_Conv2d(1, 128, 32, 32) → (1, 128, 32, 32)   147,584
005_model_encoder_resnet1_conv_bl

In [15]:
!python -m scripts.reconstruct --checkpoint outputs/ae.ckpt --data data/train_tensor_ds1.pt --num_samples 32

Using device: mps
Loading model from outputs/ae.ckpt
Hyperparameters: {'latent_dim': 32, 'lr': 3e-05, 'realtime_metrics': False, 'lambda_act': 1e-05, 'lambda_sim': 0, 'lambda_div': 0, 'out_shape': (512, 512)}
Model loaded successfully. Latent dim: 32
Test output stats: mean=0.045495, std=0.091439, max=0.596971
Loading data from data/train_tensor_ds1.pt
Data shape: torch.Size([32, 1, 512, 512])
Generating reconstructions...
Evaluating reconstruction quality...
Original non-zero pixels: 490885
Reconstruction non-zero pixels: 8388608
Original: mean=0.043847, std=0.176167
Reconstruction: mean=0.045412, std=0.091239
Reconstruction Quality Assessment: POOR
Overall Correlation: 0.4622
Log-space Correlation: 0.4618

RECONSTRUCTION QUALITY RESULTS
Samples processed   32
MSE                 0.023077 ± 0.004995
PSNR (dB)           16.48 ± 1.03
SSIM                0.2687 ± 0.0329
Masked MSE          0.300640
Correlation         0.4622
Non-zero ratio      17.0887
Intensity ratio     1.0357
⚠️  WARN

In [ ]:
!python -m scripts.generate_embeddings --input data/train_tensor.pt --checkpoint outputs/ae.ckpt --batch_size 2048 --output outputs/embeddings.pt

Traceback (most recent call last):
  File "/Users/louisg/PycharmProjects/4DSTEM_AE_MSc/scripts/reconstruct.py", line 17, in <module>
    from models.autoencoder import Autoencoder
ModuleNotFoundError: No module named 'models'


In [16]:
import torch, numpy as np
raw = torch.load("data/train_tensor.pt")
print("raw tensor shape:", raw.shape)         # (N, 1, Qy, Qx)
N = raw.shape[0]
print("number of probe positions:", N)

# quick factor search
cands = [(f, N//f) for f in range(1, int(np.sqrt(N))+1) if N % f == 0]
print("factor pairs:", cands[:10], "…")


raw tensor shape: torch.Size([4788, 1, 64, 64])
number of probe positions: 4788
factor pairs: [(1, 4788), (2, 2394), (3, 1596), (4, 1197), (6, 798), (7, 684), (9, 532), (12, 399), (14, 342), (18, 266)] …


In [ ]:
!python scripts/stem_visualization.py data/Diffraction_SI.dm4 --output outputs/stem_vis/stem_vis.png

Loaded data shape: (4788, 512, 512)
Scan shape: (42, 114)
Pattern shape: (512, 512)
Loaded .dm4 file with shape: (4788, 512, 512)
Inferred scan shape: (42, 114)
STEM visualization saved to: outputs/stem_vis/stem_vis.png


In [ ]:
!python -m scripts.evaluate_autoencoder --checkpoint outputs/ae.ckpt --data_path data/train_tensor.pt --output_path outputs/eval/evaluation_report.png

Loading model from outputs/ae.ckpt
Loading test data from data/train_tensor.pt
Evaluating on 100 samples
Generating reconstructions...
Computing comprehensive metrics...

AUTOENCODER EVALUATION SUMMARY
Quality Assessment: FAIR
Correlation: 0.8728
Intensity Ratio: 0.1124
Non-zero Ratio: 1.3515
PSNR: 7.55 dB
MSE: 0.174508
Creating comprehensive visualization...
Evaluation report saved to: outputs/eval/evaluation_report.png
Creating STEM-specific analysis...
STEM visualization saved to: outputs/eval/evaluation_report_stem_analysis.png
STEM analysis saved to: outputs/eval/evaluation_report_stem_analysis.png


In [ ]:
python scripts/visualization/visualise_scan_latents.py --raw workspace/proj_data_ds4_l8/inputs/train_tensor_ds4_compressed.h5 --latents workspace/proj_data_ds4_l8_with_reg/embeddings/ds4_8lat_with_regs_embeddings.pt --scan 209 194 --lat_max_cols 4 --outfig workspace/proj_data_ds4_l8_with_reg/analysis_and_visualisations/latent_mosaic.png

## Dimension Reduction

In [ ]:
python scripts/analysis/umap_latent_visualization.py --embeddings workspace/proj_data_ds4_l32_with_reg/dimension_reduction/umap_reduction_data.npz --output_dir workspace/proj_data_ds4_l32_with_reg/clustering --n_neighbors 50 --min_dist 0.01

In [ ]:
python scripts/analysis/cluster_scan_visualization.py --umap_results "workspace/proj_data_ds4_l32_with_reg/clustering/analysis_new/clustering_analysis_data.npz" --raw_data "workspace/proj_data_ds4_l32/inputs/train_tensor_ds4_compressed.h5" --output_dir "workspace/proj_data_ds4_l32/visualisations/umap_analysis_nn_50_md_pointzero1/stem_vis_overlay" --scan_shape 209 194 

In [ ]:
#  dimension reduction with UMAP
python scripts/analysis/dimension_reduction.py --embeddings workspace/proj_data_ds4_l32_with_reg/embeddings/ds4_32lat_with_regs_embeddings.pt --method umap --optimize_parameters --subsample 5000

In [ ]:
# pca optimisation
python scripts/analysis/dimension_reduction.py --embeddings workspace/proj_data_ds4_l8_with_reg/embeddings/ds4_8lat_with_regs_embeddings.pt --output_dir "workspace/proj_data_ds4_l8_with_reg/analysis_and_visualisations/pca_optimisation" --method pca --pca_component_sweep "3,4,5,6,7,8" --optimize_parameters --standardize --scan_shape 209 194
# umap optimisation
python scripts/analysis/dimension_reduction.py --embeddings workspace/proj_data_ds4_l8_with_reg/embeddings/ds4_8lat_with_regs_embeddings.pt --output_dir "workspace/proj_data_ds4_l8_with_reg/analysis_and_visualisations/umap_optimisation" --method umap --optimize_parameters --standardize --scan_shape 209 194


In [ ]:
# PCA with single component count (no optimization)
python scripts/analysis/dimension_reduction.py --embeddings workspace/proj_data_ds4_l32_with_reg/embeddings/ds4_32lat_with_regs_embeddings.pt --output_dir workspace/proj_data_ds4_l32_with_reg/analysis_and_visualisation/pca_15_components --method pca --n_components 15 --standardize --scan_shape 209 194

## Clustering

In [ ]:
  # For original latent embeddings (high-dimensional)
  python scripts/analysis/optimize_clustering.py --latent_embeddings workspace/proj_data_ds4_l32/outputs/embeddings/ds4_4epoch_embeddings.npz --output_dir "workspace/proj_data_ds4_l32_with_reg/analysis_and_visualisation/pca_optimisation" --max_clusters 20 --subsample 10000 --standardize

In [ ]:
  # For UMAP/PCA reduced embeddings  
  python scripts/analysis/optimize_clustering.py \
      --reduced_data results/dimension_reduction/umap_reduction_data.npz \
      --output_dir results/reduced_clustering_optimization \
      --max_clusters 15

In [ ]:
python scripts/analysis/clustering_analysis.py --reduced_embeddings workspace/proj_data_ds4_l32_with_reg/dimension_reduction_optimisation/umap_reduction_data.npz --output_dir workspace/proj_data_ds4_l32_with_reg/clustering/analysis --method kmeans --n_clusters 4 --standardize

In [ ]:
 # Compare multiple clustering methods for latent embeddings directly from ae
    python scripts/analysis/clustering_analysis.py --latent_embeddings workspace/proj_data_ds4_l32/outputs/embeddings/ds4_4epoch_embeddings.npz --output_dir workspace/proj_data_ds4_l32/visualisations/latent_embeddings_cluster_analysis --method all --compare_methods

In [ ]:
python scripts/analysis/optimize_clustering.py --reduced_data workspace/proj_data_ds4_l8_with_reg/analysis_and_visualisations/umap_optimisation/umap_reduction_data.npz  --output_dir workspace/proj_data_ds4_l8_with_reg/analysis_and_visualisations/umap_optimisation/clustering_optimisation --clusters "3"

In [ ]:
# Clustering analysis on original latent embeddings k-means
    python scripts/analysis/clustering_analysis.py --latent_embeddings workspace/proj_data_ds4_l32/outputs/embeddings/ds4_4epoch_embeddings.npz --output_dir workspace/proj_data_ds4_l8_with_reg/analysis_and_visualisations/latent_clustering --method kmeans --n_clusters 3 --standardize

In [ ]:
# Clustering analysis on reduced embeddings k-means
    python scripts/analysis/clustering_analysis.py --reduced_embeddings workspace/proj_data_ds4_l32/visualisations/umap_analysis_nn_50_k_7/umap_data.npz --output_dir workspace/proj_data_ds4_l32/visualisations/clustering_analysis_umap_k_3 --method kmeans --n_clusters 3 --standardize

In [ ]:
python scripts/analysis/optimize_clustering.py --reduced_data workspace/proj_data_ds4_l32_with_reg/analysis_and_visualisation/pca_15_components/pca_reduction_data.npz  --output_dir workspace/proj_data_ds4_l32_with_reg/analysis_and_visualisation/pca_15_components/clustering_optimisation --max_clusters 7 --subsample 10000

SyntaxError: invalid syntax (3418114949.py, line 1)

In [ ]:
python scripts/analysis/optimize_clustering.py --umap_data workspace/proj_data_ds4_l32_with_reg/dimension_reduction_single/umap_reduction_data.npz --output_dir workspace/proj_data_ds4_l32_with_reg/clustering --max_clusters 20 --subsample 10000

# latents clustering
python scripts/analysis/optimize_clustering.py --latent_embeddings workspace/proj_data_ds4_l8_with_reg/embeddings/ds4_8lat_with_regs_embeddings.pt --output_dir workspace/proj_data_ds4_l8_with_reg/analysis_and_visualisations/latent_clustering --clusters "3" --standardize

## Overlay Visualisation

In [ ]:
 python scripts/analysis/cluster_scan_visualiza# scan, bright‑field background, every latent dim in one mosaic
python scripts/visualization/visualise_scan_latents.py --raw workspace/proj_data_ds4_l32/inputs/train_tensor_ds4_compressed.h5 --latents workspace/proj_data_ds4_l32/outputs/embeddings/ds4_4epoch_embeddings.npz --scan 194 209 --virtual bf --lat_max_cols 6 --outfig workspace/proj_data_ds4_l32/outputs/latent_mosaic.pngtion.py --umap_results workspace/proj_data_ds4_l32/visualisations/spectral_clustering_optimisation/spectral_individual_results/spectral_3/cluster_info_spectral_3_clus.json --scan_shape 209 194 --output_dir workspace/proj_data_ds4_l32/visualisations/spectral_clustering_optimisation/stem_vis_overlay --raw_data workspace/proj_data_ds4_l32/inputs/train_tensor_ds4_compressed.h5

In [ ]:
python scripts/analysis/cluster_scan_visualization.py --clustered_data workspace/proj_data_ds4_l8/analysis_and_visualisations/umap_optimisation/clustering_3/spectral_individual_results/spectral_3/cluster_labels.npy  --scan_shape 209 194 --output_dir workspace/proj_data_ds4_l8/analysis_and_visualisations/umap_optimisation/clustering_3/spectral_individual_results/spectral_3/stem_vis_overlay --raw_data workspace/proj_data_ds4_l32/inputs/train_tensor_ds4_compressed.h5

In [ ]:
python scripts/analysis/umap_latent_visualization.py \
        --embeddings embeddings/patterns_embeddings.npz \
        --output_dir results/umap_analysis \
        --n_neighbors 50 \
        --min_dist 0.01

## Polar Mapping

In [ ]:
python scripts/analysis/latent_cluster_polar_map.py --data workspace/proj_data_ds4_l32/inputs/train_tensor_ds4_compressed.h5 --dset patterns --labels workspace/proj_data_ds4_l32/visualisations/umap_analysis_nn_50_k_7/umap_data.npz --scan-shape 209 194 --config workspace/proj_data_ds4_l32/visualisations/polar_map_improved/improved_polar_config.yaml --out workspace/proj_data_ds4_l32/visualisations/polar_map_improved

In [ ]:

python scripts/analysis/latent_cluster_polar_map.py --data workspace/proj_data_ds4_l32/inputs/train_tensor_ds4_compressed.h5 --dset patterns --labels workspace/proj_data_ds4_l8/analysis_and_visualisations/umap_optimisation/clustering_3/spectral_individual_results/spectral_3/cluster_labels.npy --scan-shape 209 194 --config workspace/proj_data_ds4_l8/analysis_and_visualisations/umap_optimisation/clustering_3/spectral_individual_results/spectral_3/polar_map/polar_config.yaml --out workspace/proj_data_ds4_l8/analysis_and_visualisations/umap_optimisation/clustering_3/spectral_individual_results/spectral_3/polar_map --interactive-ks
pace

In [ ]:
python scripts/analysis/latent_cluster_polar_map.py --data workspace/proj_data_ds4_l32/inputs/train_tensor_ds4_compressed.h5 --dset patterns --labels workspace/proj_data_ds4_l8/analysis_and_visualisations/umap_optimisation/clustering_3/spectral_individual_results/spectral_3/cluster_labels.npy --scan-shape 209 194 --config workspace/proj_data_ds4_l8/analysis_and_visualisations/umap_optimisation/clustering_3/spectral_individual_results/spectral_3/polar_map/polar_config.yaml --out workspace/proj_data_ds4_l8/analysis_and_visualisations/umap_optimisation/clustering_3/spectral_individual_results/spectral_3/polar_map

In [ ]:
python -m scripts.visualization.diagrams.visualize_with_torchlens --model models.autoencoder:Autoencoder --scan-y 209 --scan-x 194 --det 256 --latent-dim 32 --out torchlens_viz

In [ ]:
# Control nesting depths and which blocks to show:
  python -m scripts.visualization.diagrams.visualize_with_torchlens --model models.autoencoder:Autoencoder --blocks-depth 1 --inside-depth 4 --focus-modules encoder.resnet1,decoder.resnet_up1 --out torchlens_viz

In [ ]:
dot -Tsvg workspace/diagrams/landscape/torchlens_final_inside_decoder_decoder_resnet_up1_clean_landscape.dot -o torchlens_final_inside_decoder_decoder_resnet_up1_clean_landscape.svg
